In [ ]:
# %load main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

    if (args.save_model):
        torch.save(model.state_dict(),"mnist_cnn.pt")
        
if __name__ == '__main__':
    main()

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        conv1=x
        x = F.max_pool2d(x, 2, 2)
        pool1=x
        x = F.relu(self.conv2(x))
        conv2=x
        x = F.max_pool2d(x, 2, 2)
        pool2=x
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        fc1 = x
        x = self.fc2(x)
        fc2 = x
        return [conv1,pool1,conv2,pool2,fc1,fc2,F.log_softmax(x, dim=1)]

In [4]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64) #args.test_batch_size, shuffle=True, **kwargs)

In [10]:
device = 'cuda'
test_loss = 0
correct = 0
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
model.load_state_dict(torch.load('mnist_cnn.pt'))
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [60]:
%save get_image.py
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        conv1=x
        x = F.max_pool2d(x, 2, 2)
        pool1=x
        x = F.relu(self.conv2(x))
        conv2=x
        x = F.max_pool2d(x, 2, 2)
        pool2=x
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        fc1 = x
        x = self.fc2(x)
        fc2 = x
        return [conv1,pool1,conv2,pool2,fc1,fc2,F.log_softmax(x, dim=1)]

device = 'cuda'
test_loss = 0
correct = 0
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
model.load_state_dict(torch.load('mnist_cnn.pt'))
model.eval()



seen_num = set()
root = os.path.join('mnist images')
with torch.no_grad():
    for data, target in test_loader:
        print(target)
        print(data.shape)
        output = model(data)
        conv1,pool1,conv2,pool2,fc1,fc2,_ = output
        print(conv1[0].shape, conv2[0].shape, fc1[0].shape, fc2[0].shape)
        target = list(target.numpy())
        for idx,label in enumerate(target):
            if label not in seen_num:
                seen_num.add(label)
                image_path=os.path.join(root,str(label))
                print(image_path)
                #===============================================================
                input_image = data[i][0]
                input_image = np.array(np.uint8(input_image),dtype=float)
                input_image = Image.fromarray(input_image)
                input_imgray = input_image.convert('L')
                input_image = np.array(list(input_imgray.getdata()), float)
                input_image.shape = (input_imgray.size[1], input_imgray.size[0])
                input_image = (input_image/2)*255
                input_image = np.round(input_image)
                numpy.save(os.path.join(image_path,'input','input.npy'), input_image)
                input_image = Image.fromarray(input_image)
                input_image = pil.convert('RGB')
                input_image.save(os.path.join(image_path,'input','input.png'))
                #================================================================
                image_conv1 = conv1[i]
                i=0
                image = None
                input_image=None
                for i,image in enumerate(image_conv1):
                    input_image = np.array(np.uint8(image),dtype=float)
                    input_image = Image.fromarray(input_image)
                    input_imgray = input_image.convert('L')
                    input_image = np.array(list(input_imgray.getdata()), float)
                    input_image.shape = (input_imgray.size[1], input_imgray.size[0])
                    #print(input_image[np.argmax(input_image,axis=1)])
                    #max_val = input_image(np.argmax(input_image))
                    max_value =np.amax(input_image)
#                     if max_value == 0:
#                         print(input_image)
#                         assert(max_value!=0)
                    #print(a)
                    input_image = (input_image/max_value)*255
                    input_image = np.round(input_image)
                    numpy.save(os.path.join(image_path,'conv1','pickles','feature_map{}.npy'.format(i)),input_image)
                    #print(input_image)
                    input_image = Image.fromarray(input_image)
                    input_image = pil.convert('RGB')
                    input_image.save(os.path.join(image_path,'conv1','imgs','feature_map{}.png'.format(i)))
                #================================================================
                image_pool1 = pool1[i]
                i=0
                image = None
                input_image=None
                for i,image in enumerate(image_pool1):
                    input_image = np.array(np.uint8(image),dtype=float)
                    input_image = Image.fromarray(input_image)
                    input_imgray = input_image.convert('L')
                    input_image = np.array(list(input_imgray.getdata()), float)
                    input_image.shape = (input_imgray.size[1], input_imgray.size[0])
                    max_value =np.amax(input_image)
                    input_image = (input_image/max_value)*255
                    input_image = np.round(input_image)
                    numpy.save(os.path.join(image_path,'pool1','pickles','feature_map{}.npy'.format(i)),input_image)
                    input_image = Image.fromarray(input_image)
                    input_image = pil.convert('RGB')
                    input_image.save(os.path.join(image_path,'pool1','imgs','feature_map{}.png'.format(i)))
                #=================================================================
                image_conv2 = conv2[i]
                i=0
                image = None
                input_image=None
                for i,image in enumerate(image_conv2):
                    input_image = np.array(np.uint8(image),dtype=float)
                    input_image = Image.fromarray(input_image)
                    input_imgray = input_image.convert('L')
                    input_image = np.array(list(input_imgray.getdata()), float)
                    input_image.shape = (input_imgray.size[1], input_imgray.size[0])
                    max_value =np.amax(input_image)
                    input_image = (input_image/max_value)*255
                    input_image = np.round(input_image)
                    numpy.save(os.path.join(image_path,'conv2','pickles','feature_map{}.npy'.format(i)),input_image)
                    input_image = Image.fromarray(input_image)
                    input_image = pil.convert('RGB')
                    input_image.save(os.path.join(image_path,'conv2','imgs','feature_map{}.png'.format(i)))
                #=================================================================
                image_pool2 = pool2[i]
                i=0
                image = None
                input_image=None
                for i,image in enumerate(image_conv2):
                    input_image = np.array(np.uint8(image),dtype=float)
                    input_image = Image.fromarray(input_image)
                    input_imgray = input_image.convert('L')
                    input_image = np.array(list(input_imgray.getdata()), float)
                    input_image.shape = (input_imgray.size[1], input_imgray.size[0])
                    max_value =np.amax(input_image)
                    input_image = (input_image/max_value)*255
                    input_image = np.round(input_image)
                    numpy.save(os.path.join(image_path,'pool2','pickles','feature_map{}.npy'.format(i)),input_image)
                    input_image = Image.fromarray(input_image)
                    input_image = pil.convert('RGB')
                    input_image.save(os.path.join(image_path,'pool2','imgs','feature_map{}.png'.format(i)))
                #==================================================================
                image = None
                input_image=None
                image=fc1[i]
                
                input_image = np.array(np.uint8(image),dtype=float)
                input_image = Image.fromarray(np.array([input_image]))
                input_imgray = input_image.convert('L')
                input_image = np.array(list(input_imgray.getdata()), float)
                input_image.shape = (input_imgray.size[1], input_imgray.size[0])
                max_value =np.amax(input_image)
                input_image = (input_image/max_value)*255
                input_image = np.round(input_image)
                numpy.save(os.path.join(image_path,'fc1','pickles','feature_map{}.npy'.format(i)),input_image)
                input_image = Image.fromarray(input_image)
                input_image = pil.convert('RGB')
                input_image.save(os.path.join(image_path,'fc1','imgs','feature_map{}.png'.format(i)))
                #====================================================================
                image = None
                input_image=None
                image=fc2[i]
                input_image = np.array(np.uint8(image),dtype=float)
                input_image = Image.fromarray(np.array([input_image]))
                input_imgray = input_image.convert('L')
                input_image = np.array(list(input_imgray.getdata()), float)
                input_image.shape = (input_imgray.size[1], input_imgray.size[0])
                max_value =np.amax(input_image)
                input_image = (input_image/max_value)*255
                input_image = np.round(input_image)
                numpy.save(os.path.join(image_path,'fc1','pickles','feature_map{}.npy'.format(i)),input_image)
                input_image = Image.fromarray(input_image)
                input_image = pil.convert('RGB')
                input_image.save(os.path.join(image_path,'fc1','imgs','feature_map{}.png'.format(i)))
            else:
                continue

'' was not found in history, as a file, url, nor in the user namespace.
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3])
torch.Size([64, 1, 28, 28])


ValueError: too many values to unpack (expected 7)

TypeError: 'DataLoader' object does not support indexing

In [11]:

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)


    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



Test set: Average loss: 0.0307, Accuracy: 9896/10000 (99%)

